# Project 1: Classification For German Bank Credit Risk

### Part 1 - Data Cleaning

#### By: Amin
For this notebook, I will be cleaning the dataset & do an initial exploratory data analysis.  
<hr>

**About Dataset**

This dataset consist of 1000 records of individuals with 9 features + target.  

Features:  

1. Age (numeric)
2. Sex (text: male, female)
3. Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
4. Housing (text: own, rent, or free)
5. Saving accounts (text - little, moderate, quite rich, rich)
6. Checking account (numeric, in DM - Deutsch Mark)
7. Credit amount (numeric, in DM)
8. Duration (numeric, in month)
9. Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

Data Source: https://www.kaggle.com/uciml/german-credit

## Objectives

1) Determine the client profile of those with and without a good or bad risk based on features available.  

2) Create a model that would be able to predict whether an individual is a good or bad risk.  

## 1. Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 2. Getting our Data

In [2]:
df = pd.read_csv('credit.csv')
df.head()

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


## 3. Initial EDA

In [3]:
df.shape

(1000, 11)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   int64 
 1   Age               1000 non-null   int64 
 2   Sex               1000 non-null   object
 3   Job               1000 non-null   int64 
 4   Housing           1000 non-null   object
 5   Saving accounts   817 non-null    object
 6   Checking account  606 non-null    object
 7   Credit amount     1000 non-null   int64 
 8   Duration          1000 non-null   int64 
 9   Purpose           1000 non-null   object
 10  Risk              1000 non-null   object
dtypes: int64(5), object(6)
memory usage: 86.1+ KB


In [5]:
df.isnull().sum()

Unnamed: 0            0
Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

- Savings accounts: 183 rows of null values
- Checking account: 394 rows of null values

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,1000.0,499.500,288.819436,0.0,249.75,499.5,749.25,999.0
Age,1000.0,35.546,11.375469,19.0,27.00,33.0,42.00,75.0
Job,1000.0,1.904,0.653614,0.0,2.00,2.0,2.00,3.0
Credit amount,1000.0,3271.258,2822.736876,250.0,1365.50,2319.5,3972.25,18424.0
Duration,1000.0,20.903,12.058814,4.0,12.00,18.0,24.00,72.0


### 3.2 Renaming Column & Setting it as Index

In [7]:
df.rename(columns = {'Unnamed: 0':'id'}, inplace = True)
df.set_index('id', inplace=True)
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
id,,,,,,,,,,
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


### 3.3 Looking into each Variables

In [8]:
df['Risk'].value_counts(normalize=True)*100

good    70.0
bad     30.0
Name: Risk, dtype: float64

In [9]:
df['Purpose'].value_counts(normalize=True)*100

car                    33.7
radio/TV               28.0
furniture/equipment    18.1
business                9.7
education               5.9
repairs                 2.2
domestic appliances     1.2
vacation/others         1.2
Name: Purpose, dtype: float64

In [10]:
df['Sex'].value_counts(normalize=True)*100

male      69.0
female    31.0
Name: Sex, dtype: float64

In [11]:
df['Job'].value_counts(normalize=True)*100

2    63.0
1    20.0
3    14.8
0     2.2
Name: Job, dtype: float64

In [12]:
df['Housing'].value_counts(normalize=True)*100

own     71.3
rent    17.9
free    10.8
Name: Housing, dtype: float64

In [13]:
df['Duration'].value_counts().head(10)

24    184
12    179
18    113
36     83
6      75
15     64
9      49
48     48
30     40
21     30
Name: Duration, dtype: int64

In [14]:
df['Checking account'].value_counts(normalize=True)*100

little      45.214521
moderate    44.389439
rich        10.396040
Name: Checking account, dtype: float64

In [15]:
df['Saving accounts'].value_counts(normalize=True)*100

little        73.806610
moderate      12.607099
quite rich     7.711138
rich           5.875153
Name: Saving accounts, dtype: float64

### Observations:

1) <strong>Risk</strong>: We have an imbalanced data where good risks are at 70% of the data while bad risks are at 30%  

2) <strong>Purpose</strong>: The purchase of cars, radio/TV & furniture/equipment makes up the majority for the purpose. Repairs, domestic appliances and vacation/others makes up a small percentage as the bottom 3 purpose. I may consider grouping them up as "others"  

3) <strong>Sex</strong>: Males make up the majority for this dataset at 69% while females make up 31%  

4) <strong>Job</strong>: Skilled labour makes up the majority at 63%, followed by unskilled & resident at 20%, highly skilled at 14.8% and lastly, unskilled & non-resident at 2.2%  

5) <strong>Housing</strong>: Most lives in their own housing at 71.3%, followed by renting at 17.9% and lastly those living in free housing at 10.8%  

6) <strong>Duration</strong>: The duration of loans varies greatly, but the majority hovers around multiples of 6 and/or 12 months.  

7) <strong>Checking Account</strong>: The majority have little to moderate amount in their checking account at around 45% each while a smaller percentage, 10%, are rich. I will be filling up the null values here with the <em>mode</em>.  

8) <strong>Saving Accounts</strong>: The majority has little savings in their account at around 74%, moderate at around 13%, quite rich at around 8% and lastly rich at around 6%. I will be filling up the null values here with the <em>mode</em>.  


## 4. Cleaning the Data

### 4.1 Replace Null Values

In [16]:
df['Checking account'].replace(np.nan, 'little', inplace=True)
df['Saving accounts'].replace(np.nan, 'little', inplace=True)

### 4.2 Tidying up the Columns

In [17]:
def cleaned_col(column_names):
    cleaned_cols = []
    for c in column_names:
        c = c.lower().replace(' ', '_')
        cleaned_cols.append(c)
    return cleaned_cols

In [18]:
df.columns = cleaned_col(df.columns)

In [19]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,risk
id,,,,,,,,,,
0,67,male,2,own,little,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,little,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


## 5. Saving the Cleaned Data for EDA

In [20]:
df.to_csv('cleaned_data.csv', index = False)